## Extraction and Load into Staging Database


### Extraction

In [ ]:
from src.extract.extraction import extract_database,extract_api,extract_spreadsheet

In [ ]:
# database
sales_df = extract_database(table_name='car_sales', engine_name='source')
sales_df.head()

,id_sales,year,brand_car,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,1,2014,Chevrolet,Impala Limited,LT Fleet,Sedan,automatic,2g1wb5e37e1112559,fl,4.0,21507.0,white,black,gm remarketing,13450.0,13800.0,Mon Feb 23 2015 05:00:00 GMT-0800 (PST)
1,2,2003,Dodge,Ram Pickup 1500,SLT,Quad Cab,,1d7ha18n13s152972,mo,31.0,79712.0,—,black,tdaf remarketing,6025.0,6300.0,Tue Jan 20 2015 02:30:00 GMT-0800 (PST)
2,3,2007,Pontiac,G6,GT,Convertible,automatic,1g2zh361474252178,nj,34.0,65698.0,red,black,car authority inc,7375.0,8000.0,Wed Jan 14 2015 01:30:00 GMT-0800 (PST)
3,4,2011,Toyota,Corolla,LE,Sedan,automatic,jtdbu4eexb9167571,fl,43.0,23634.0,black,beige,world omni financial corporation,10800.0,11400.0,Tue Jan 27 2015 01:30:00 GMT-0800 (PST)
4,5,2012,Lexus,ES 350,Base,Sedan,,jthbk1eg6c2495519,pa,35.0,26483.0,black,brown,meridian remarketing,22500.0,23300.0,Fri Jan 30 2015 01:00:00 GMT-0800 (PST)


In [ ]:
state_df = extract_api(url="https://raw.githubusercontent.com/Kurikulum-Sekolah-Pacmann/us_states_data/refs/heads/main/us_states.json")
state_df.head()

,id_state,code,name
0,1,al,Alabama
1,2,ak,Alaska
2,3,az,Arizona
3,4,ar,Arkansas
4,5,ca,California


In [ ]:
brand_df = extract_spreadsheet(worksheet_name='brand_car')
brand_df.head()

,brand_car_id,brand_name,created_at
1,1,Acura,2025-11-27 20:33:20
2,2,Audi,2025-11-27 20:33:20
3,3,Bentley,2025-11-27 20:33:20
4,4,BMW,2025-11-27 20:33:20
5,5,Buick,2025-11-27 20:33:20


### Load into Staging Databasse

In [ ]:
from src.load.load import load_database

#### car sales

In [ ]:
sales_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_sales      30000 non-null  int64  
 1   year          30000 non-null  int64  
 2   brand_car     30000 non-null  object 
 3   model         30000 non-null  object 
 4   trim          30000 non-null  object 
 5   body          30000 non-null  object 
 6   transmission  30000 non-null  object 
 7   vin           30000 non-null  object 
 8   state         30000 non-null  object 
 9   condition     29341 non-null  float64
 10  odometer      29993 non-null  float64
 11  color         30000 non-null  object 
 12  interior      30000 non-null  object 
 13  seller        30000 non-null  object 
 14  mmr           29998 non-null  float64
 15  sellingprice  30000 non-null  float64
 16  saledate      30000 non-null  object 
dtypes: float64(4), int64(2), object(11)
memory usage: 3.9+ MB


In [ ]:
# subset columns and set index
list_sales_columns = ['id_sales','year','brand_car','transmission','state','condition','odometer','color','interior','mmr','sellingprice']
sales_df = sales_df[list_sales_columns]
sales_df = sales_df.set_index('id_sales')


In [ ]:
# load into staging
load_database(df=sales_df, table_name='car_sales', engine_name='staging')

#### us_state

In [ ]:
state_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id_state  68 non-null     int64 
 1   code      68 non-null     object
 2   name      68 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.7+ KB


In [ ]:
state_df = state_df.set_index('id_state')

In [ ]:
load_database(df=state_df, table_name='us_state', engine_name='staging')

#### car_brand

In [ ]:
brand_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 1 to 51
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand_car_id  51 non-null     object
 1   brand_name    51 non-null     object
 2   created_at    51 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [ ]:
brand_df = brand_df.set_index('brand_car_id')

In [ ]:
load_database(df=brand_df, table_name='car_brand', engine_name='staging')

## ETL into Warehouse Database

### Extraction

In [1]:
import pandas as pd
from src.extract.extraction import extract_database

In [2]:
# database
sales_df = extract_database(table_name='car_sales', engine_name='staging')
sales_df.head()

,id_sales,year,brand_car,transmission,state,condition,odometer,color,interior,mmr,sellingprice,created_at
0,1,2014,Chevrolet,automatic,fl,4.0,21507.0,white,black,13450.0,13800.0,2025-11-29 09:20:44.454350
1,2,2003,Dodge,,mo,31.0,79712.0,—,black,6025.0,6300.0,2025-11-29 09:20:44.454350
2,3,2007,Pontiac,automatic,nj,34.0,65698.0,red,black,7375.0,8000.0,2025-11-29 09:20:44.454350
3,4,2011,Toyota,automatic,fl,43.0,23634.0,black,beige,10800.0,11400.0,2025-11-29 09:20:44.454350
4,5,2012,Lexus,,pa,35.0,26483.0,black,brown,22500.0,23300.0,2025-11-29 09:20:44.454350


In [3]:
brand_df = extract_database(table_name='car_brand', engine_name='staging')
brand_df.head()

,brand_car_id,brand_name,created_at
0,1,Acura,2025-11-29 16:20:44
1,2,Audi,2025-11-29 16:20:44
2,3,Bentley,2025-11-29 16:20:44
3,4,BMW,2025-11-29 16:20:44
4,5,Buick,2025-11-29 16:20:44


In [4]:
state_df = extract_database(table_name='us_state', engine_name='staging')
state_df.head()

,id_state,code,name,created_at
0,1,al,Alabama,2025-11-29 09:20:53.444181
1,2,ak,Alaska,2025-11-29 09:20:53.444181
2,3,az,Arizona,2025-11-29 09:20:53.444181
3,4,ar,Arkansas,2025-11-29 09:20:53.444181
4,5,ca,California,2025-11-29 09:20:53.444181


In [ ]:
len(sales_df)

30000

### Transform

- After exploration data there are some datas that need to be transformed:
  - merging data
  - change data type for several columns
  - rename column 
  - subset columns for warehouse db


In [ ]:
# Merge data
sales_brand_df = pd.merge(sales_df,brand_df,left_on='brand_car', right_on='brand_name',how='left')
merge_df = pd.merge(sales_brand_df,state_df,left_on='state', right_on='code',how='left')

In [ ]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id_sales      30000 non-null  int64         
 1   year          30000 non-null  object        
 2   brand_car     30000 non-null  object        
 3   transmission  30000 non-null  object        
 4   state         30000 non-null  object        
 5   condition     29341 non-null  object        
 6   odometer      29993 non-null  object        
 7   color         30000 non-null  object        
 8   interior      30000 non-null  object        
 9   mmr           29998 non-null  object        
 10  sellingprice  30000 non-null  object        
 11  created_at_x  30000 non-null  datetime64[ns]
 12  brand_car_id  29474 non-null  float64       
 13  brand_name    29474 non-null  object        
 14  created_at_y  29474 non-null  datetime64[ns]
 15  id_state      29998 non-null  float6

In [ ]:
# list of columns to use in warehouse
list_colums = ['id_sales','year','brand_car_id','transmission','id_state','condition','odometer','color','interior','mmr','sellingprice']
merge_df = merge_df[list_colums]

#rename columns 
columns = {
    'id_sales':'id_sales_nk',
    'sellingprice':'selling_price'
}
merge_df = merge_df.rename(columns=columns)

In [ ]:
merge_df.isna().sum()

id_sales_nk        0
year               0
brand_car_id     526
transmission       0
id_state           2
condition        659
odometer           7
color              0
interior           0
mmr                2
selling_price      0
dtype: int64

We can give brand_car_id, and id_state null values with 0 for now, because it won't be used later for modelling

In [ ]:
merge_df['brand_car_id'] = merge_df['brand_car_id'].fillna(0)
merge_df['id_state'] = merge_df['id_state'].fillna(0)

In [ ]:
list_col = ['year','transmission','condition','odometer','color','interior','mmr','selling_price']
for col in list_col:
    print(f"Column: {col}")
    print(merge_df[col].unique())
    print("===================================")

Column: year
['2014' '2003' '2007' '2011' '2012' '2005' '2013' '2002' '2008' '2015'
 '2006' '2009' '1998' '2004' '2010' '2000' '2001' '1999' '1996' '1991'
 '1994' '1993' '1995' '1997' '1988' '1992' '1989' '1985' '1987']
Column: transmission
['automatic' '' 'manual' 'Sedan']
Column: condition
['4.0' '31.0' '34.0' '43.0' '35.0' '32.0' '49.0' '42.0' '21.0' '28.0'
 '38.0' '26.0' '47.0' '46.0' '25.0' '24.0' '29.0' '37.0' '44.0' '19.0'
 '36.0' '23.0' '27.0' '48.0' '22.0' '39.0' None '1.0' '41.0' '33.0' '3.0'
 '45.0' '5.0' '2.0' '17.0' '11.0' '16.0' '14.0' '15.0' '13.0' '18.0'
 '12.0']
Column: odometer
['21507.0' '79712.0' '65698.0' ... '41092.0' '292925.0' '25083.0']
Column: color
['white' '—' 'red' 'black' 'burgundy' 'blue' 'silver' 'purple' '' 'gray'
 'gold' 'beige' 'green' 'charcoal' 'yellow' 'brown' 'orange' 'off-white'
 'turquoise' '16633' 'pink' '6388']
Column: interior
['black' 'beige' 'brown' 'tan' 'gray' '' '—' 'silver' 'green' 'off-white'
 'blue' 'red' 'burgundy' 'gold' 'purple' 'o

In [ ]:
cols_to_cast = {
    'year':'int',
    'odometer':'float',
    'condition':'float',
    'mmr':'float',
    'selling_price':'float',
    'id_state':'int',
    'brand_car_id':'int'
}

merge_df = merge_df.astype(cols_to_cast)

In [ ]:
merge_df.head()

,id_sales_nk,year,brand_car_id,transmission,id_state,condition,odometer,color,interior,mmr,selling_price
0,1,2014,7,automatic,9,4.0,21507.0,white,black,13450.0,13800.0
1,2,2003,10,,25,31.0,79712.0,—,black,6025.0,6300.0
2,3,2007,38,automatic,30,34.0,65698.0,red,black,7375.0,8000.0
3,4,2011,48,automatic,9,43.0,23634.0,black,beige,10800.0,11400.0
4,5,2012,27,,38,35.0,26483.0,black,brown,22500.0,23300.0


In [5]:
from src.transform.transform import transform_data  

In [6]:
merge_df = transform_data(sales_df,brand_df,state_df)
merge_df.head()

,year,brand_car_id,transmission,id_state,condition,odometer,color,interior,mmr,selling_price
id_sales_nk,,,,,,,,,,
1,2014,7,automatic,9,4.0,21507.0,white,black,13450.0,13800.0
2,2003,10,,25,31.0,79712.0,—,black,6025.0,6300.0
3,2007,38,automatic,30,34.0,65698.0,red,black,7375.0,8000.0
4,2011,48,automatic,9,43.0,23634.0,black,beige,10800.0,11400.0
5,2012,27,,38,35.0,26483.0,black,brown,22500.0,23300.0


In [ ]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 1 to 30000
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           30000 non-null  int64  
 1   brand_car_id   30000 non-null  int64  
 2   transmission   30000 non-null  object 
 3   id_state       30000 non-null  int64  
 4   condition      29341 non-null  float64
 5   odometer       29993 non-null  float64
 6   color          30000 non-null  object 
 7   interior       30000 non-null  object 
 8   mmr            29998 non-null  float64
 9   selling_price  30000 non-null  float64
dtypes: float64(4), int64(3), object(3)
memory usage: 2.5+ MB


### Load 


In [ ]:
from src.load.load import load_database

In [ ]:
load_database(df=merge_df, table_name='car_sales', engine_name='warehouse')

## Modelling

In [7]:
from src.extract.extraction import extract_database
import numpy as np

In [8]:
car_df = extract_database(table_name='car_sales', engine_name='warehouse')
car_df.head()

,sales_id,id_sales_nk,year,brand_car_id,transmission,id_state,condition,odometer,color,interior,mmr,selling_price,created_at
0,6107ef38-d092-420e-ab9e-88120259ab7c,1066,2004,24,,5,27.0,119146.0,blue,gray,1750.0,1300.0,2025-11-29 09:46:40.930373
1,c6ce798d-1273-43a8-982f-7311fec5bd57,1919,2010,35,,38,4.0,41825.0,blue,gray,8325.0,8100.0,2025-11-29 09:46:40.930373
2,c4d81934-fb08-4da3-81d2-8dc0691654b6,3883,2005,23,,42,2.0,99125.0,beige,—,5550.0,2800.0,2025-11-29 09:46:40.930373
3,dd73adc0-157b-4aa6-96bd-2b0192fede7d,3981,2004,0,,61,27.0,169674.0,green,gray,2450.0,1350.0,2025-11-29 09:46:40.930373
4,96112b8c-3582-4c22-b7fa-9a38f70b3537,4351,2000,0,,38,33.0,141872.0,blue,gray,1775.0,1600.0,2025-11-29 09:46:40.930373


### Cleaning Data

In [9]:
from src.modelling.clean import cleaning_data

In [10]:
clean_df = cleaning_data(car_df)
clean_df.head()

,year,transmission,condition,odometer,color,interior,mmr,selling_price
0,2004,NaN,27.0,119146.0,blue,gray,1750.0,1300.0
1,2010,NaN,4.0,41825.0,blue,gray,8325.0,8100.0
2,2005,NaN,2.0,99125.0,beige,NaN,5550.0,2800.0
3,2004,NaN,27.0,169674.0,green,gray,2450.0,1350.0
4,2000,NaN,33.0,141872.0,blue,gray,1775.0,1600.0


### Preprocessing

In [11]:
import pandas as pd

from src.modelling.preprocessing.splitting_data import split_data

In [12]:
# Split data
X_train,X_test,y_train,y_test = split_data(clean_df, target='selling_price', test_size=0.2, random_state=42)

In [13]:
X_train.head()

,year,transmission,condition,odometer,color,interior,mmr
12190,2012,NaN,41.0,27338.0,black,black,12500.0
775,2013,automatic,4.0,56241.0,gray,NaN,36600.0
21229,2012,automatic,49.0,24017.0,silver,gray,22600.0
22380,2011,automatic,44.0,95971.0,black,gray,16400.0
26826,2010,NaN,42.0,40158.0,black,black,23300.0


In [14]:
from src.modelling.preprocessing.preprocess import preprocess_data

In [15]:
num_cols = ['year','odometer','condition','mmr']
cat_cols = ['transmission','color','interior']

In [16]:
preprocess = preprocess_data(num_cols,cat_cols)

X_train_pre = preprocess.fit_transform(X_train)
X_test_pre = preprocess.transform(X_test)

### XGBoost

In [17]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
)

model.fit(X_train_pre, y_train)

y_train_pred = model.predict(X_train_pre)


mse = mean_squared_error(y_train, y_train_pred)
rmse = mse ** 0.5
print(rmse)



1149.5873140431747


In [18]:
import numpy as np

mape = round(np.mean(np.abs((y_train - y_train_pred) / y_train)) * 100, 2)
print(mape)

11.94


In [19]:
# test
X_test_pre = preprocess.transform(X_test)
y_test_pred = model.predict(X_test_pre)


mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = mse_test ** 0.5
print(rmse_test)


2019.755133946973


In [20]:
mape = round(np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100, 2)
print(mape)

13.41


In [21]:
from src.modelling.xgboost import modelling_process

modelling_process(X_train_pre, y_train, X_test_pre, y_test)

Train RMSE: 1149.59
Train MAPE: 11.94
Test RMSE: 2019.76
Test MAPE: 13.41
